In [1]:
import numpy as np
import numpy.linalg as npl
import scipy.linalg as spl
import scipy.integrate as integrate
import matplotlib.pyplot as plt

### Résolution du problème 1D, avec conditions de Dirichlet homogènes

In [40]:
#Mesh definition
xm = 0
xM = 1

ym = 0
yM = 1
y0 = 1/2

N = 2*50 # 100 points selon x, 50 sur y sous la disc et 50 sur y sur la disc

h = (xM - xm) / (N)

MeshM = np.meshgrid(np.linspace(xm, xM, N), np.linspace(y0, yM, int(N/2)))
Meshm = np.meshgrid(np.linspace(xm, xM, N), np.linspace(ym, y0, int(N/2)))

InM = (MeshM[0][ 1:-1, 1:-1], MeshM[1][ 1:-1, 1:-1])
Inm = (Meshm[0][ 1:-1, 1:-1], Meshm[1][ 1:-1, 1:-1])
# x = np.linspace(xm, xM,N)
# ym = np.linspace(ym,y0,int(N/2))[:-1]
# yM = np.linspace(y0, yM, int(N/2))[1:]

In [41]:
#function definition
def a(x, y):
    return 1

def f(x):
    return np.exp(-(x-1/2)**2)

Beta = 1    

U = np.zeros((N, N))

In [42]:
def quadrature(f, a, b, c): # A TESTER

    # print(a[0].shape)
    # print(a[1].shape)
    # print(b[0].shape)
    # print(b[1].shape)
    # print(c[0].shape)
    # print(c[1].shape)

    c1 = (a[0] - b[0])**2 + (a[1] - b[1])**2
    c2 = (a[0] - c[0])**2 + (a[1] - c[1])**2
    c3 = (b[0] - c[0])**2 + (b[1] - c[1])**2
    p = c1 + c2 + c3      #perimetre
    aire = np.sqrt(p*(p - c1)*(p - c2)*(p - c3)) 

    p1 = f((a[0] + b[0] + 3*c[0])/5, (a[1] + b[1] + 3*c[1])/5)
    p2 = f((a[0] + 3*b[0] + c[0])/5, (a[1] + 3*b[1] + c[1])/5)
    p3 = f((3*a[0] + b[0] + c[0])/5, (3*a[1] + b[1] + c[1])/5)
    
    
    intval = (25/48) * (p1 + p2 + p3) * aire

    return intval

In [43]:
# def quadrature(f, a, b, c): # A TESTER

#     c1 = npl.norm(a-b, 2)
#     c2 = npl.norm(a-c, 2)
#     c3 = npl.norm(c-b, 2)
#     p = c1 + c2 + c3      #perimetre
#     aire = np.sqrt(p*(p - c1)*(p - c2)*(p - c3)) 

#     p1 = f((a + b + 3*c)/5)
#     p2 = f((a + 3*b + c)/5)
#     p3 = f((3*a + b + c)/5)
    
    
#     intval = (25/48) * (p1 + p2 + p3) * aire
    
#     return intval

In [44]:
#calcul du produit scalaire quand i==j, cad sur la diagonale

def ScalarProductOverlap(f, M_x, M_y, h):  # f function (a), M1 M2 les points autours des quels Phi_i et Phi_j sont centrés.
    
    T = np.zeros(M_x[ 1:-1, 1:-1].shape)
    # right = np.array([h, 0])
    # high = np.array([0, h])

    # print(quadrature(f, M,  M + left, M + left + down )
    # T+= quadrature(f, M,  M + left, M + left + down ) #T1 : centre , gauche et bas gauche
    # T+= quadrature(f, M,  M + down , M + left + down ) #T2 : centre , bas et diagonale basse
    # T+= quadrature(f, M,  M + down, M + right )         #T3 : centre , bas et droite
    # T+= quadrature(f, M,  M + right, M + right + high ) #T4 : centre , haut droit et droite
    # T+= quadrature(f, M,  M + high, M + right + high )  #T5 : centre , haut droit et haut
    # T+= quadrature(f, M,  M + left, M + high )         #T6 : centre , haut et gauche
    M = np.array([M_x, M_y])

    T+= quadrature(f, (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 1:-1, :-2], M_y[ 1:-1, :-2]), (M_x[2:,:-2], M_y[2:,:-2])) #T1 : centre , gauche et bas gauche
    T+= quadrature(f, (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 2:, 1:-1], M_y[ 2:, 1:-1]), (M_x[2:,:-2], M_y[2:,:-2])) #T2 : centre , bas et bas gauche
    T+= quadrature(f, (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 2:, 1:-1], M_y[ 2:, 1:-1]), (M_x[1:-1,2:], M_y[1:-1,2:])) #T3 : centre , bas et droite
    T+= quadrature(f, (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ :-2, 2:], M_y[ :-2, 2:]), (M_x[1:-1,2:], M_y[1:-1,2:])) #T4 : centre , haut droit et droite
    T+= quadrature(f, (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ :-2, 2:], M_y[ :-2, 2:]), (M_x[:-2,1:-1], M_y[:-2,1:-1])) #T5 : centre , haut droit et haut
    T+= quadrature(f, (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 1:-1, :-2], M_y[ 1:-1, :-2]), (M_x[:-2,1:-1], M_y[:-2,1:-1]))  #T6 : centre , gauche et haut

    return T

In [45]:
def ScalarProductSideOverlap(f, M_x, M_y, h, side):  # f function (a), M1 M2 les points autours des quels Phi_i et Phi_j sont centrés.
    T = 0

    if side=="right":
        T+= quadrature(lambda x, y : -f(x, y), (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ :-2, 2:], M_y[ :-2, 2:]), (M_x[1:-1,2:], M_y[1:-1,2:]))  #T4 : centre , droite et haut droit
        T+= quadrature(lambda x, y : -f(x, y), (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 2:, 1:-1], M_y[ 2:, 1:-1]), (M_x[1:-1,2:], M_y[1:-1,2:])) #T3 : centre , bas et droite
    elif side=="left":
        T+= quadrature(lambda x, y : -f(x, y), (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 1:-1, :-2], M_y[ 1:-1, :-2]), (M_x[2:,:-2], M_y[2:,:-2])) #T1 : centre , gauche et bas gauche
        T+= quadrature(lambda x, y : -f(x, y), (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 1:-1, :-2], M_y[ 1:-1, :-2]), (M_x[:-2,1:-1], M_y[:-2,1:-1]))  #T6 : centre , gauche et haut


    return T

In [46]:
def ScalarProductVerticalOverlap(f, M_x, M_y, h, side):  # f function (a), M1 M2 les points autours des quels Phi_i et Phi_j sont centrés.
    T = 0

    if side=="haut":
        T+= quadrature(lambda x, y : -f(x, y), (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ :-2, 2:], M_y[ :-2, 2:]), (M_x[:-2,1:-1], M_y[:-2,1:-1])) #T5 : centre , haut droit et haut
        T+= quadrature(lambda x, y : -f(x, y), (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 1:-1, :-2], M_y[ 1:-1, :-2]), (M_x[:-2,1:-1], M_y[:-2,1:-1]))  #T6 : centre , gauche et haut

    elif side=="bas":
        T+= quadrature(lambda x, y : -f(x, y), (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 2:, 1:-1], M_y[ 2:, 1:-1]), (M_x[2:,:-2], M_y[2:,:-2])) #T2 : centre , bas et bas gauche
        T+= quadrature(lambda x, y : -f(x, y), (M_x[ 1:-1, 1:-1], M_y[ 1:-1, 1:-1]), (M_x[ 2:, 1:-1], M_y[ 2:, 1:-1]), (M_x[1:-1,2:], M_y[1:-1,2:])) #T3 : centre , bas et droite
    

    return T

In [63]:
def Mat_Sys(Meshm, MeshM):

        
    InM = (MeshM[0][ 1:-1, 1:-1], MeshM[1][ 1:-1, 1:-1])
    Inm = (Meshm[0][ 1:-1, 1:-1], Meshm[1][ 1:-1, 1:-1])
    
    Nx = MeshM[0].shape[1]
    Ny = MeshM[0].shape[0]

    h = (xM - xm) / (N + 1)

    C = np.zeros(( (Nx - 2) * (2*Ny - 1), (Nx - 2) * (2*Ny - 1)))
    print(C.shape)
    #print( C[:(Nx - 2)*(Ny - 2), :(Nx - 2)*(Ny - 2)].shape)
    #print(C[(Nx - 2)*(Ny - 2) + 2*98:, (Nx - 2)*(Ny - 2) + 2*98:].shape)

    #partie continue haut de la matrice donc bas du maillage: 
    upperblock = np.zeros(( (Nx - 2) * (Ny - 2), (Nx - 2) * (Ny - 2)))
    Centre =  ScalarProductOverlap(a, Meshm[0], Meshm[1], h)/(h**2)
    upperblock += np.diag(np.ravel(Centre))

    diag1 = ScalarProductSideOverlap(a, Meshm[0], Meshm[1], h, "right")/(h**2) 
    diag_1 =  ScalarProductSideOverlap(a, Meshm[0], Meshm[1], h, "left")/(h**2) 

    upperblock += np.diag(np.ravel( diag1 )[:-1], 1)  #il faut enlever ceux calculé en trop sur les cotés mais bizarre
    upperblock += np.diag(np.ravel( diag_1 )[1:], -1)  #idem 

    diag2 = ScalarProductVerticalOverlap(a, Meshm[0], Meshm[1], h, "haut")/(h**2) 
    diag_2 = ScalarProductVerticalOverlap(a, Meshm[0], Meshm[1], h, "bas")/(h**2) 

    #print(len(np.ravel( diag2 )[:-1]))
    #print(np.diag(np.ravel( diag2 )[:-1], N).shape)

    upperblock += np.diag(np.ravel( diag2 )[:-N], N)   #il faut enlever ceux calculé en trop sur le haut et le bas
    upperblock += np.diag(np.ravel( diag_2 )[N:], -N)  #idem 

    C[:(Nx - 2)*(Ny - 2), :(Nx - 2)*(Ny - 2)] = upperblock

    #partie continue basse :
    lowerblock = np.zeros(( (Nx - 2) * (Ny - 2), (Nx - 2) * (Ny - 2)))

    Centre =  ScalarProductOverlap(a, MeshM[0], MeshM[1], h)/(h**2)               
    lowerblock += np.diag(np.ravel(Centre))
    
    diag1 = ScalarProductSideOverlap(a, MeshM[0], MeshM[1], h, "right")/(h**2)
    diag_1 =  ScalarProductSideOverlap(a,MeshM[0], MeshM[1], h, "left")/(h**2)
    lowerblock += np.diag(np.ravel( diag1 )[:-1], 1)
    lowerblock += np.diag(np.ravel( diag_1 )[1:], -1)

    diag2 = ScalarProductVerticalOverlap(a, MeshM[0], MeshM[1], h, "haut")/(h**2) 
    diag_2 = ScalarProductVerticalOverlap(a, MeshM[0], MeshM[1], h, "bas")/(h**2) 

    #print(len(np.ravel( diag2 )[:-1]))
    #print(np.diag(np.ravel( diag2 )[:-1], N).shape)

    lowerblock += np.diag(np.ravel( diag2 )[:-(N-2)], N-2)   #je sais pas comment il calcule le dernier en tout cas le dernier n'a pas de sens
    lowerblock += np.diag(np.ravel( diag_2 )[N-2:], -N+2)  #idem pour le premier

    C[(Nx - 2)*(Ny - 2) + 3*98:, (Nx - 2)*(Ny - 2) + 3*98:] += lowerblock
    
    #discontinuité il reste a remplir une croix au milieu de la matrice




    

    return C

In [62]:
print(Mat_Sys(Meshm, MeshM))

(9702, 9702)
[[ 0.00862184 -0.00287395  0.         ...  0.          0.
   0.        ]
 [-0.00287395  0.00862184 -0.00287395 ...  0.          0.
   0.        ]
 [ 0.         -0.00287395  0.00862184 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.00862184 -0.00287395
   0.        ]
 [ 0.          0.          0.         ... -0.00287395  0.00862184
  -0.00287395]
 [ 0.          0.          0.         ...  0.         -0.00287395
   0.00862184]]


In [ ]:
test = np.arange(100)
print(np.diag(test, 100).shape)

(200, 200)


In [12]:
def phi(n, m ,x, y): # à utiliser uniquement loin de la discontinuité 

    if y<y0:
        return 
    else:
        return



    if y<y0:
        return (x > Meshm[n-1, ])*(x < Meshm[n])*(x-Meshm[n-1])/(Meshm[n]-Meshm[n-1]) + (x > Meshm[n])*(x < Meshm[n+1])*(Meshm[n+1]-x)/(Meshm[n+1]-Meshm[n])
    else:
        return (x > MeshM[n-1])*(x < MeshM[n])*(x-MeshM[n-1])/(MeshM[n]-MeshM[n-1]) + (x > MeshM[n])*(x < MeshM[n+1])*(MeshM[n+1]-x)/(MeshM[n+1]-MeshM[n])



def Sec_Mem(Nm,NM):
    fm = np.zeros(Nm+1)
    fM = np.zeros(NM+1)
    
    #i = np.linspace(1,Nm+1,Nm+1)
    #fm[:] = integrate.quad(lambda x :f(x)*phi(i[:],x),0,1)[0]    -> même pb que plus haut
    #fM[:] = integrate.quad(lambda x :f(x)*phi(i[:],x),0,1)[0]
    for i in range(Nm+1):
        fm[i] = integrate.quad(lambda x :f(x)*phi(i,x),Meshm[i],Meshm[i+1])[0]
    for i in range(NM+1):
        fM[i] = integrate.quad(lambda x :f(x)*phi(i,x),MeshM[i],MeshM[i+1])[0]
    fm[Nm] = integrate.quad(lambda x :f(x)*phi(Nm,x),0,x0)[0]
    fM[0] = integrate.quad(lambda x :f(x)*phi(0,x),x0,1)[0]

    return np.concatenate((fm,fM))


In [13]:
U[1:-1] = spl.solve(Mat_Sys(Nm,NM),Sec_Mem(Nm,NM))
plt.plot(Meshm,U[:Nm+2])
plt.plot(MeshM,U[Nm+2:])
plt.show()

NameError: name 'Nm' is not defined